# Monte-Carlo Learning
* Model-Free Learning으로, State transition Probability와 Reward를 모를 때 학습할 수 있는 기법을 의미한다.
## Evaluation
* $V^{\pi}(s_t)=E_{\pi}[G_t\mid S_t=s_t]$에서 $G_t^{\pi}=R_{t+1}^{\pi}+\gamma R_{t+2}^{\pi}+\cdots+\gamma^{T-1}R_{t+T}^{\pi}$는
* Expectation에 의해 밖으로 꺼내면 $R_{t+1}^{\pi}=P_{ss'}^{\pi}R_{t+1}$이 되는데, 이 때 State transition probability $P_{ss'}^{\pi}$와 Reward $R_{t+1}$를 알 수가 없다.
* * *
* 이에 대해 결론적으로, $V^{\pi}(s_t)$를 구하는 것이 목적이기 때문에 이는 Law of Large Numbers로 구할 수 있다.
* 무한 번에 가까운 에피소드를 발생시켜서 나오는 무한 개에 가까운 Value function의 평균을 구하면 모든 State의 Value function을 구할 수 있다.
* 이 과정에서 State transition probability를 모르는 건 상관이 없다. <u>현재의 Policy에 의존하여 <b>추론</b>을 할 것이기 때문에 괜찮다.</u>>
* * * 
* 또한, Value function 값을 계속 누적으로 가지고 있는 것은 비효율적이기 때문에 Incremental Mean을 사용해서 Episode가 끝나면 즉각적으로 평균값에 업데이트한다.
## Improvement
* 하지만, Policy를 update(Improvement)하려면 Action Value가 필요한데 Evaluation에서는 State Value를 구하는 것을 채택했다.
* State Value로부터 Action Value를 도출하는 방법이 있지만, 이것은 Reward와 State transition probability를 필요로 한다.
* 이것은 알 수가 없기 때문에, 애초에 Evaluation 할 때부터 <u>State Value가 아니라 Action Value를 구해버리면 된다.</u>
    * 16개의 State와 4개의 Action이 있다고 했을 때, 16개 구할 거 64개 구하는 것과 같다.

In [1]:
import numpy as np

In [4]:
class GridWorld:
    '''
    Environment, Grid 4x4
    '''
    def __init__(self):
        self.agent_pos = {
            'y': 0,
            'x': 0
        }
        
        self.goal_pos = {
            'y': 3,
            'x': 3
        }
        
        self.y_min, self.x_min, self.y_max, self.x_max = 0, 0, 3, 3
        
        # 해당 코드에서는 State가 4x4 matrix 그 자체라 보면된다.
        self.state = np.zeros([4, 4]) # 4x4 Grid
        # 그래서 아래는 Initial state라고 보면 됨.
        self.state[self.agent_pos['y'], self.agent_pos['x']] = 1
        
        # 모든 state에 대한 저장
        self.state_space = list()
        for y in range(4):
            for x in range(4):
                state = np.zeros([4, 4])
                state[y, x] = 1
                self.state_space.append(state)
        
        self.action_space = [0, 1, 2, 3] # U, D, L, R
        self.gamma = 0.9
        
    
    def reset(self):
        self.agent_pos = {
            'y': 0,
            'x': 0
        }
        self.state = np.zeros([4, 4])
        self.state[self.agent_pos['y'], self.agent_pos['x']] = 1
        
        return self.state
    
    def step(self, action):
        '''
        Args:
            action: [0, 3] 범위의 int 값으로 어떤 Action을 취하는지 의미한다.
        Return:
            reward: s->a->s'에 대한 Reward
            self.state: (list) 다음 state(4x4)
            done: (bool) Goal state에 도달했는가?
        '''
        
        # Action에 따라 state를 이동하면서 Boundary를 넘지 않도록 한다.
        if action == 0:
            self.agent_pos['y'] = max(self.agent_pos['y']-1, self.y_min)
        elif action == 1:
            self.agent_pos['y'] = min(self.agent_pos['y']+1, self.y_max)
        elif action == 2:
            self.agent_pos['x'] = max(self.agent_pos['x']-1, self.x_min)
        elif action == 3:
            self.agent_pos['x'] = min(self.agent_pos['x']+1, self.x_max)
        else:
            raise AssertionError("Invalid Action")
        
        # State transition(self.state 갱신) 및 Get reward
        prev_state = self.state
        self.state = np.zeros([4, 4])
        self.state[self.agent_pos['y'], self.agent_pos['x']] = 1
        reward = self.reward(prev_state, action, self.state)
        
        # Episode done?
        done = True if (self.agent_pos == self.goal_pos) else False
        
        return reward, self.state, done
    
    def reward(self, s, a, s_next):
        reward = 0
        y, x = np.where(s == 1)
        y_next, x_next = np.where(s_next == 1)
        
        # State transition 했을 때, Goal state인가?
        if (
            (y_next == self.goal_pos['y'] and x_next == self.goal_pos['x']) and
            (y != self.goal_pos['y'] or x != self.goal_pos['x'])
            ):
            reward = 10
            
        return reward

    def get_state_index(self, state_space, state):
        # State에 해당하는 index가 무엇인가?
        for i_s, s in enumerate(state_space):
            if (s == state).all():
                return i_s
        raise AssertionError("Couldn\'t find the state from the state space")
    
    def exploring_start(self):
        # Initial State를 랜덤하게 선택
        while True:
            y_random = np.random.randint(4)
            x_random = np.random.randint(4)
            self.agent_pos = {
                'y': y_random,
                'x': x_random
            }
            if self.agent_pos != self.goal_pos:
                break
        
        self.state = np.zeros([4, 4])
        self.state[self.agent_pos['y'], self.agent_pos['x']] = 1
        return self.state

In [11]:
def calc_return(gamma, rewards):
    n = len(rewards)
    rewards = np.array(rewards)
    gammas = gamma * np.ones([n])
    powers = np.arange(n)
    
    power_of_gammas = np.power(gammas, powers)
    discounted_rewards = rewards * power_of_gammas
    g = np.sum(discounted_rewards)
    
    return g

def mc_control(env, policy):
    action_value_matrix = np.zeros([len(env.state_space), len(env.action_space)]) # 16 x 4
    returns = [[{'n':0, 'avg':0} for a in env.action_space] for s in env.state_space]
    
    for loop_count in range(1000):
        episode = {
            'states': [],
            'actions': [],
            'rewards': []
        }
        done = False
        step_count = 0
        s = env.exploring_start() # Initial state 초기화
        
        # Generate and episode
        while not done:
            s_inx = env.get_state_index(env.state_space, s)
            pi_s = policy[s_inx]
            a = np.random.choice(env.action_space, p=pi_s) # Policy의 확률을 통해 랜덤으로 선택
            r, s_next, done = env.step(a)
            
            episode['states'].append(s)
            episode['actions'].append(a)
            episode['rewards'].append(r)
            
            step_count += 1
            s = s_next
            
            is_dead_lock = False
            if step_count > 10000:
                is_dead_lock = True
                break
        
        if is_dead_lock: # Episode가 끝나지 않는 Deadlock epsidoe였다면, evalutaion에서 제외한다.
            continue
        
        episode['states'].append(s) # Goal state append
        
        # Policy Evaluation (State-Action)
        for t in range(step_count):
            s_t = episode['states'][t]
            a_t = episode['actions'][t]
            i_s_t = env.get_state_index(env.state_space, s_t)
            i_a_t = env.action_space.index(a_t)
            g_t = calc_return(env.gamma, episode['rewards'][t:])
            
            n_prev, avg_prev = returns[i_s_t][i_a_t]['n'], returns[i_s_t][i_a_t]['avg']
            # Incremental Mean (각 State 별로)
            returns[i_s_t][i_a_t]['avg'] = (avg_prev * n_prev + g_t) / (n_prev + 1)
            
            returns[i_s_t][i_a_t]['n'] = n_prev + 1
            action_value_matrix[i_s_t][i_a_t] = returns[i_s_t][i_a_t]['avg']

        # Policy Improvement (MC도 Episode 당 한 번)
        # 그런데, 굳이 Episode 단위로 도는 이유가 있나? 여러 번 똑같은 Update하는 거일텐데?
        for t in range(step_count):
            s_t = episode['states'][t]
            i_s_t = env.get_state_index(env.state_space, s_t)
            
            a_max = action_value_matrix[i_s_t].argmax()
            policy[i_s_t][:] = 0
            policy[i_s_t][a_max] = 1
            
        if (loop_count + 1) % 10 == 0:
            print(f"[{loop_count+1}] action_value_matrix: \n{action_value_matrix}")
            
    return policy, action_value_matrix

In [13]:
env = GridWorld()

# Policy 초기화
policy = list()
for i_s, s in enumerate(env.state_space):
    pi = np.array([0.25]*4)
    policy.append(pi)
policy = np.array(policy)

policy, action_value_matrix = mc_control(env, policy)

value_vector = np.sum(policy * action_value_matrix, axis=-1)
value_table = value_vector.reshape(4, 4)

[10] action_value_matrix: 
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.94440943e-01 5.21637808e-03 0.00000000e+00 6.32140694e-01]
 [6.08768095e-01 4.32042586e+00 3.23456233e-01 3.52077918e-03]
 [2.32740674e-01 0.00000000e+00 1.64037850e-01 1.09644108e-01]
 [0.00000000e+00 1.03565192e-01 0.00000000e+00 8.31299428e-02]
 [0.00000000e+00 1.33921294e-02 8.64135386e-02 3.70939007e+00]
 [3.32989637e-03 0.00000000e+00 1.47808829e-01 6.29738434e+00]
 [3.09031544e-01 7.57949865e+00 0.00000000e+00 3.29429710e+00]
 [7.93470378e-02 3.94635133e-02 6.95164702e-02 3.00632632e+00]
 [4.41390838e+00 0.00000000e+00 2.53814215e-02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 7.51714430e+00]
 [2.82429536e+00 1.00000000e+01 5.43085975e+00 4.88777377e+00]
 [2.68583708e+00 0.00000000e+00 3.21232232e-02 3.75710213e-02]
 [2.97332279e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [8.10000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e

In [14]:
policy

array([[0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  , 1.  ],
       [0.  , 1.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  , 1.  ],
       [0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 1.  ],
       [1.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 1.  ],
       [0.  , 1.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  , 0.  ],
       [0.25, 0.25, 0.25, 0.25]])